In [1]:
import sys, os

from tqdm import tqdm
tqdm.pandas()

from dotenv import load_dotenv
import pandas as pd

load_dotenv("../.env")

True

In [2]:
df = pd.read_csv("../data/bigbasket_loreal.csv")
df.head(2)

,index,product,category,sub_category,brand,sale_price,market_price,type,rating,description
0,935,6 Oil Nourish Conditioner,Beauty & Hygiene,Hair Care,Loreal Paris,159.0,199.0,Shampoo & Conditioner,4.3,Does your hair get frizzy and unmanageable? Ar...
1,984,Men Expert White Activ Intensive Whitening Foam,Beauty & Hygiene,Men's Grooming,Loreal Paris,499.0,499.0,Face & Body,3.0,LOreal Men Expert White Activ intensive whiten...


In [3]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=1.0)

In [4]:
from langchain_core.messages import HumanMessage

def get_questions(product_name, product_description):
    questions = chat.invoke(
    [
        HumanMessage(
            content=f"""Your job is to generate questions for the product descriptions such that it is hard to answer the question.


Example 1
Product name: Fructis Serum - Long & Strong
Product description: Garnier Fruits Long & Strong Strengthening Serum detangles unruly hair, softens hair without heaviness and helps stop split and breakage ends. It is enriched with the goodness of Grape seed and avocado oil that result in smoother, shinier and longer hair.  
Questions: 
- Does Garnier Fruits Long & Strong Strengthening Serum help in hair growth?
- Which products contain avocado oil?

Example 2
Product name: Color Naturals Creme Riche Ultra Hair Color - Raspberry Red
Product description: Garnier Color Naturals is creme hair colour which gives 100% Grey Coverage and ultra visible colour with 50% more shine. It has a superior Colour Lock technology which gives you a rich long-lasting colour that lasts up to 8 weeks. Color Naturals comes in a range of 8 gorgeous shades especially suited for Indian skin tones. It is available in an easy to use kit which can be used at your convenience in the comfort of your house! It is enriched with the goodness of 3 oils - Almond, Olive and Avocado which nourishes hair and provides shiny, long-lasting colour. Your hair will love the nourishment and you will love the colour!
Questions: 
- How long does Color Naturals Creme Riche Ultra Hair Color last?
- Which product for hair color is suited for indian skin?
- How many colors are available in Color Naturals Hair Color?

Product name: Black Naturals Hair Colour Shade 1-Deep Black 20 ml + 20 gm
Product description: It is an oil-enriched cream colour which gives natural-looking black hair. Works in 15 minutes, non-drip cream. Maintains softness, smoothness, and shine. No ammonia hair colour. Lasts for 6 weeks.
Questions: 
- Does Black Naturals Hair Colour contain ammonia?
- How muct time do you have to keep Grey Naturals Hair Colour on your hair?

Now generate 5 questions which can be answered for this product based on description. Use the exact product name in the questions as mentioned in the description. There should not be duplicates in the 5 questions. Return questions starting with - instead of numbers.

Product name: {product_name}
Product description: {product_description}
Questions: """
        )
    ]
)
    questions = questions.content.replace("- ", "").split("\n")
    questions = list(filter(None, questions)) 
    return questions

product_name, product_description = df.iloc[10][["product", "description"]]
print(product_name, "\n", product_description)
get_questions(product_name, product_description)

Men Expert Hydra Energetic Cleanser 
 L'Oreal Paris Men Expert Hydra Energetic Cleanser is enriched with cryo-tonic. This is a soap-free pH neutral formula and it deeply cleanses and purifies skin without drying it out. its active defense system has a powerful soothing active ingredient that helps strengthen the skin's natural resistance against daily aggressions. Prevents skin tightness. Refreshes and vitalizes the skin. Deeply cleanses and purifies


["Does L'Oreal Paris Men Expert Hydra Energetic Cleanser contain any soap?",
 "How does the active defense system in L'Oreal Paris Men Expert Hydra Energetic Cleanser work?",
 "What is the pH level of L'Oreal Paris Men Expert Hydra Energetic Cleanser?",
 "What is the main ingredient in cryo-tonic of L'Oreal Paris Men Expert Hydra Energetic Cleanser?",
 "How does L'Oreal Paris Men Expert Hydra Energetic Cleanser prevent skin tightness?"]

In [5]:
df["questions"] = df.progress_apply(lambda x: get_questions(x["product"], x["description"]), axis=1)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [05:47<00:00,  3.48s/it]


In [6]:
df.head()

,index,product,category,sub_category,brand,sale_price,market_price,type,rating,description,questions
0,935,6 Oil Nourish Conditioner,Beauty & Hygiene,Hair Care,Loreal Paris,159.00,199.0,Shampoo & Conditioner,4.3,Does your hair get frizzy and unmanageable? Ar...,[Is 6 Oil Nourish Conditioner for all hair typ...
1,984,Men Expert White Activ Intensive Whitening Foam,Beauty & Hygiene,Men's Grooming,Loreal Paris,499.00,499.0,Face & Body,3.0,LOreal Men Expert White Activ intensive whiten...,[Does LOreal Men Expert White Activ Intensive ...
2,990,Men Expert Hydra Power Duo Foam Cleanser,Beauty & Hygiene,Skin Care,Loreal Paris,367.54,499.0,Face & Body,4.1,"Detoxifying Anti-Roughness, the foam deeply cl...",[How does Men Expert Hydra Power Duo Foam Clea...
3,1074,White Perfect - Clinical New Skin Essence Lotion,Beauty & Hygiene,Skin Care,Loreal Paris,995.00,995.0,Face Care,NaN,L'Oreal Paris White Perfect Clinical New Skin ...,[How does L'Oreal Paris White Perfect Clinical...
4,1202,Revitalift Moisturizing Day Cream SPF 23 PA++,Beauty & Hygiene,Skin Care,Loreal Paris,622.39,845.0,Face Care,4.8,L'Oreal Paris Revitalift Moisturizing Day Crea...,[What elements is L'Oreal Paris Revitalift Moi...


In [7]:
df.to_csv("../data/bigbasket_loreal.csv", index=False)